# Mosaico 2021

Script trabajado por Francisco Albornoz (2021-04-13) / Última modificación / Mosaicos Patagonia

2021-08-05 13:07:09 

In [1]:
import ee
import geemap
import geemap.chart as chart
import os
import folium
import xarray_leaflet
import rasterio 
from rasterio import plot
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

<img src=https://www.ecopaysen.cl/wp-content/uploads/2019/12/ecopaysen_marca_web.png>

In [2]:
Map = geemap.Map(center = [-45, -70], zoom = 4)
Map1 = geemap.Map(center = [-45, -70], zoom = 4)
Map2 = geemap.Map(center = [-45, -70], zoom = 4)
Map3 = geemap.Map(center = [-45, -70], zoom = 4)
Map4 = geemap.Map(center = [-45, -70], zoom = 4)
Map5 = geemap.Map(center = [-45, -70], zoom = 4)
Map6 = geemap.Map(center = [-45, -70], zoom = 4)
Map7 = geemap.Map(center = [-45, -70], zoom = 4)
Map8 = geemap.Map(center = [-45, -70], zoom = 4)
Map9 = geemap.Map(center = [-45, -70], zoom = 4)
Map10 = geemap.Map(center = [-45, -70], zoom = 4)
Map11 = geemap.Map(center = [-45, -70], zoom = 4)
Map12 = geemap.Map(center = [-45, -70], zoom = 4)
Map13 = geemap.Map(center = [-45, -70], zoom = 4)
Map14 = geemap.Map(center = [-45, -70], zoom = 4)
Map15 = geemap.Map(center = [-45, -70], zoom = 4)

## Área de Estudio

In [3]:
work_dir = os.path.expanduser('../../../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia_geo.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)
base1 = os.path.join(work_dir, 'base_geo.shp')
base = geemap.shp_to_ee(base1)

In [4]:
base2 = os.path.join(work_dir, 'base2.shp')
base2 = geemap.shp_to_ee(base2)
base3 = os.path.join(work_dir, 'base3.shp')
base3 = geemap.shp_to_ee(base3)
base4 = os.path.join(work_dir, 'base4.shp')
base4 = geemap.shp_to_ee(base4)
base5 = os.path.join(work_dir, 'base5.shp')
base5 = geemap.shp_to_ee(base5)
base6 = os.path.join(work_dir, 'base6.shp')
base6 = geemap.shp_to_ee(base6)

In [5]:
geometria_base = os.path.join(work_dir, 'mapa_base_dis.shp') 
geometria_base = geemap.shp_to_ee(geometria_base)

In [6]:
geometria_base2 = os.path.join(work_dir, 'area_base2_dis.shp') 
geometria_base2 = geemap.shp_to_ee(geometria_base2)

## Colors Bars

In [7]:
import geemap.colormaps as cm

In [8]:
# cm.palettes.terrain

In [9]:
color_ndvi = cm.palettes.ndvi

In [10]:
color_ndwi = cm.palettes.ndwi

In [11]:
color_terrain = cm.palettes.terrain

In [12]:
color_glaciar = cm.palettes.PuBuGn_r

In [13]:
color_nubes = cm.palettes.inferno

In [14]:
# cm.list_colormaps()

In [15]:
# cm.plot_colormaps(width=12, height=0.4)

## Funciones

In [16]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [17]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [18]:
def addNDVI2(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI2')
    return image.addBands(ndvi)

In [19]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B3','B5']).rename('NDWI')
    return image.addBands(ndwi)

In [20]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B3','B4']).rename('NDGI')
    return image.addBands(ndgi)

In [21]:
def addSAVI(image):
    savi = image.expression(
        '((b5 - b4)/(b5 + b4 + 0.5))*1.5', {
            'b4':image.select('B4'),
            'b5':image.select('B5')
        }) 
    savi= savi.rename('SAVI')
    return image.addBands(savi)

In [22]:
def addNDSI(image):
    ndsi = image.normalizedDifference(['B3','B6']).rename('NDSI')
    return image.addBands(ndsi)

In [23]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [24]:
def Cloud40(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(40)) 

In [25]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [26]:
def Cloud20(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(20)) 

In [27]:
def Cloud10(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(10))

In [28]:
def addCloud(image):
    nube = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.addBands(ee.Image(nube.select(['cloud'])).rename('nube'))

In [29]:
def find_closest(target_image, image_col, days):
    """Filter images in a collection by date proximity and spatial intersection to a target image.

    Args:
        target_image: An ee.Image whose observation date is used to find near-date images in
          the provided image_col image collection. It must have a 'system:time_start' property.
        image_col: An ee.ImageCollection to filter by date proximity and spatial intersection
          to the target_image. Each image in the collection must have a 'system:time_start'
          property.
        days: A number that defines the maximum number of days difference allowed between
          the target_image and images in the image_col.

    Returns:
        An ee.ImageCollection that has been filtered to include those images that are within the
          given date proximity to target_image and intersect it spatially.
    """

    # Compute the timespan for N days (in milliseconds).
    range = ee.Number(days).multiply(1000 * 60 * 60 * 24)

    filter = ee.Filter.And(
        ee.Filter.maxDifference(range, 'system:time_start', None, 'system:time_start'),
        ee.Filter.intersects('.geo', None, '.geo'))

    closest = (ee.Join.saveAll('matches', 'measure')
        .apply(ee.ImageCollection([target_image]), image_col, filter))

    return ee.ImageCollection(ee.List(closest.first().get('matches')))

In [30]:
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [31]:
vis_nieve = {
  'bands': ['B6', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

## Colecciones

### 2021 Verano

In [32]:
start_date = '2020-11-01'
end_date = '2021-03-31'

l8_2021 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)  

In [33]:
coleccion2021 = l8_2021.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [51]:
NDVI_2021 = coleccion2021.qualityMosaic('NDVI')
NDWI_2021 = coleccion2021.qualityMosaic('NDWI')
SAVI_2021 = coleccion2021.qualityMosaic('SAVI')
NDGI_2021 = coleccion2021.qualityMosaic('NDGI')
NDSI_2021 = coleccion2021.qualityMosaic('NDSI')

In [34]:
c2021_sn = coleccion2021.map(Cloud10)
NDVI_2021_sn = c2021_sn.qualityMosaic('NDVI')
NDWI_2021_sn = c2021_sn.qualityMosaic('NDWI')
SAVI_2021_sn = c2021_sn.qualityMosaic('SAVI')
NDGI_2021_sn = c2021_sn.qualityMosaic('NDGI')
NDSI_2021_sn = c2021_sn.qualityMosaic('NDSI')

In [53]:
Map.addLayer(NDVI_2021_sn, vizParams, 'QMsn de NDVI 2021')
Map.addLayer(NDVI_2021, vizParams, 'QM de NDVI 2021')
# Map.addLayer(NDWI_2021_sn, vizParams, 'QMsn de NDWI 2021')
# Map.addLayer(SAVI_2021_sn, vizParams, 'QMsn de SAVI 2021')
# Map.addLayer(NDGI_2021_sn, vis_nieve, 'QMsn de NDGI 2021')
# Map.addLayer(NDSI_2021_sn, vis_nieve, 'QMsn de NDSI 2021')
Map

Map(bottom=2923.0, center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=H…

In [54]:
NDVI_2021_l8 = NDVI_2021.set('SENSOR_ID', 'OLI_TIRS')

In [55]:
cloud_2021_l8 = ee.Algorithms.Landsat.simpleCloudScore(NDVI_2021_l8)
cloud_2021_l8 = cloud_2021_l8.select('cloud')

In [56]:
palette_nubes = ['#1a9850', '#66bd63', '#a6d96a', '#d9ef8b', '#fee08b', '#fdae61', '#f46d43', '#d73027']

In [57]:
resultados= [cloud_2021_l8, NDVI_2021]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes 2021',
    'QM NDVI 2021']

In [58]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

### 2020 Verano

In [36]:
start_date = '2019-11-01'
end_date = '2020-03-31'

l8_2020 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

### 2019 Verano

In [37]:
start_date = '2018-11-01'
end_date = '2019-03-31'

l8_2019 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

### 2018 Verano

In [38]:
start_date = '2017-11-01'
end_date = '2018-03-31'

l8_2018 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

### 2017 Verano

In [39]:
start_date = '2016-11-01'
end_date = '2017-03-31'

l8_2017 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

### 2021-2017 Colección 

In [40]:
coleccion2021_2017 = ee.ImageCollection(l8_2021.merge(l8_2020).merge(l8_2019).merge(l8_2018).merge(l8_2017))

In [41]:
l8_summer= coleccion2021_2017.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

### Tier 1


In [42]:
Tier1_sn = l8_summer.map(Cloud10)

In [43]:
QM_NDVI_T1 = Tier1_sn.qualityMosaic('NDVI')# Normalized difference vegetation Index
QM_NDWI_T1 = Tier1_sn.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_T1 = Tier1_sn.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_T1 = Tier1_sn.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_T1 = Tier1_sn.qualityMosaic('NDSI') # Normalized difference snow index

In [44]:
Map2.addLayer(QM_NDVI_T1, vizParams, 'QM de NDVI T1')
# Map2.addLayer(QM_NDWI_T1, vizParams, 'QM de NDWI T1')
# Map2.addLayer(QM_SAVI_T1, vizParams, 'QM de SAVI T1')
# Map2.addLayer(QM_NDGI_T1, vis_nieve, 'QM de NDGI T1')
# Map2.addLayer(QM_NDSI_T1, vis_nieve, 'QM de NDSI T1')
Map2 

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Tier 2

In [45]:
Tier2_sn = l8_summer.map(Cloud20)

In [46]:
QM_NDVI_T2 = Tier2_sn.qualityMosaic('NDVI')# Normalized difference vegetation Index
QM_NDWI_T2 = Tier2_sn.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_T2 = Tier2_sn.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_T2 = Tier2_sn.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_T2 = Tier2_sn.qualityMosaic('NDSI') # Normalized difference snow index

In [47]:
# Map2.addLayer(QM_NDVI_T2, vizParams, 'QM de NDVI T2')
# Map2.addLayer(QM_NDWI_T2, vizParams, 'QM de NDWI T2')
# Map2.addLayer(QM_SAVI_T2, vizParams, 'QM de SAVI T2')
# Map2.addLayer(QM_NDGI_T2, vis_nieve, 'QM de NDGI T2')
# Map2.addLayer(QM_NDSI_T2, vis_nieve, 'QM de NDSI T2')
# Map2 

### Tier 3

In [48]:
Tier3_sn = l8_summer.map(Cloud30)

In [77]:
QM_NDVI_T3 = Tier3_sn.qualityMosaic('NDVI')# Normalized difference vegetation Index
QM_NDWI_T3 = Tier3_sn.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_T3 = Tier3_sn.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_T3 = Tier3_sn.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_T3 = Tier3_sn.qualityMosaic('NDSI') # Normalized difference snow index

### Tier 4

In [49]:
Tier4_sn = l8_summer.map(Cloud40)

In [78]:
QM_NDVI_T4 = Tier4_sn.qualityMosaic('NDVI')# Normalized difference vegetation Index
QM_NDWI_T4 = Tier4_sn.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_T4 = Tier4_sn.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_T4 = Tier4_sn.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_T4 = Tier4_sn.qualityMosaic('NDSI') # Normalized difference snow index

### 2021

#### Img base

##### Muestras de imagenes del mosaico

In [44]:
start_date = '2020-10-01'
end_date = '2021-03-31'
imgbase = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(base) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER') \
    .first() 


In [45]:
imgbase.propertyNames().getInfo()

['system:version',
 'system:id',
 'RADIANCE_MULT_BAND_5',
 'RADIANCE_MULT_BAND_6',
 'RADIANCE_MULT_BAND_3',
 'RADIANCE_MULT_BAND_4',
 'RADIANCE_MULT_BAND_1',
 'RADIANCE_MULT_BAND_2',
 'K2_CONSTANT_BAND_11',
 'K2_CONSTANT_BAND_10',
 'system:footprint',
 'REFLECTIVE_SAMPLES',
 'SUN_AZIMUTH',
 'CPF_NAME',
 'DATE_ACQUIRED',
 'ELLIPSOID',
 'google:registration_offset_x',
 'google:registration_offset_y',
 'STATION_ID',
 'RESAMPLING_OPTION',
 'ORIENTATION',
 'WRS_ROW',
 'RADIANCE_MULT_BAND_9',
 'TARGET_WRS_ROW',
 'RADIANCE_MULT_BAND_7',
 'RADIANCE_MULT_BAND_8',
 'IMAGE_QUALITY_TIRS',
 'TRUNCATION_OLI',
 'CLOUD_COVER',
 'GEOMETRIC_RMSE_VERIFY',
 'COLLECTION_CATEGORY',
 'GRID_CELL_SIZE_REFLECTIVE',
 'CLOUD_COVER_LAND',
 'GEOMETRIC_RMSE_MODEL',
 'COLLECTION_NUMBER',
 'IMAGE_QUALITY_OLI',
 'LANDSAT_SCENE_ID',
 'WRS_PATH',
 'google:registration_count',
 'PANCHROMATIC_SAMPLES',
 'PANCHROMATIC_LINES',
 'GEOMETRIC_RMSE_MODEL_Y',
 'REFLECTIVE_LINES',
 'TIRS_STRAY_LIGHT_CORRECTION_SOURCE',
 'GEOMETRIC_

In [46]:
# imgbase.get('LANDSAT_PRODUCT_ID').getInfo()

In [47]:
# imgbase.get('system:time_start').getInfo()

In [48]:
imgbase.get('DATUM').getInfo()

'WGS84'

In [49]:
imgbase.get('ELLIPSOID').getInfo()

'WGS84'

In [50]:
imgbase.get('MAP_PROJECTION').getInfo()

'UTM'

In [51]:
imgbase.get('UTM_ZONE').getInfo()

19

In [52]:
# print(imgbase.projection())

In [53]:
# time_start = imgbase.get('system:time_start')

In [54]:
imgbase = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_231091_20210130')

In [55]:
base_pro = imgbase.select('B4').projection().getInfo()

In [56]:
print(base_pro)

{'type': 'Projection', 'crs': 'EPSG:32619', 'transform': [30, 0, 214185, 0, -30, -4825185]}


In [57]:
# Map1.addLayer(imgbase, vizParams, 'img base')
# Map1.addLayer(geometria_base2)
# Map1

In [58]:
# reproj= imgbase \
#     .reproject(crs='EPSG:3857',scale = 30)\
#     .reduceResolution(reducer= ee.Reducer.mean(),bestEffort=True, maxPixels = 65536)

In [59]:
resampled = imgbase.resample('bicubic')

In [60]:
landsat_pro = resampled.select('B4').projection().getInfo()

In [61]:
escala = resampled.select('B4').projection().nominalScale().getInfo()

In [62]:
print(landsat_pro)

{'type': 'Projection', 'crs': 'EPSG:32619', 'transform': [30, 0, 214185, 0, -30, -4825185]}


In [63]:
print(escala)

30


In [64]:
# print(imgbase.projection())

In [65]:
imgbase.get('MAP_PROJECTION').getInfo()

'UTM'

In [66]:
start_date = '2020-10-01'
end_date = '2021-03-31'
imgbase2 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filter(ee.Filter.eq('WRS_PATH', 230)) \
    .filter(ee.Filter.eq('WRS_ROW', 91)) \
    .filterDate(start_date, end_date) 

In [67]:
imgbase2 = find_closest(imgbase, imgbase2,14).sort('CLOUD_COVER').first()
imgbase2 = ee.Image(imgbase2)

In [68]:
# reproj2= imgbase2 \
#     .reproject(crs='EPSG:4326',crsTransform= [0.00025, 0, -180, 0, -0.00025, 80])

In [69]:
# Map1.addLayer(imgbase2, vizParams, 'img base2')
# Map1

In [70]:
start_date = '2020-10-01'
end_date = '2021-03-31'
imgbase3 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filter(ee.Filter.eq('WRS_PATH', 229)) \
    .filter(ee.Filter.eq('WRS_ROW', 91)) \
    .filterDate(start_date, end_date) 

In [71]:
imgbase3 = find_closest(imgbase2, imgbase3,21).sort('CLOUD_COVER').first()
imgbase3 = ee.Image(imgbase3)

In [72]:
# reproj3= imgbase3 \
#     .reproject(crs='EPSG:4326',crsTransform= [0.00025, 0, -180, 0, -0.00025, 80])

In [73]:
# Map1.addLayer(imgbase3, vizParams, 'img base3')
# Map1

In [74]:
start_date = '2020-10-01'
end_date = '2021-03-31'
imgbase4 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filter(ee.Filter.eq('WRS_PATH', 228)) \
    .filter(ee.Filter.eq('WRS_ROW', 91)) \
    .filterDate(start_date, end_date) 

In [75]:
imgbase4 = find_closest(imgbase3, imgbase4,14).sort('CLOUD_COVER').first()
imgbase4 = ee.Image(imgbase4)

In [76]:
# reproj4= imgbase4 \
#     .reproject(crs='EPSG:4326',crsTransform= [0.00025, 0, -180, 0, -0.00025, 80])

In [77]:
# Map1.addLayer(imgbase4, vizParams, 'img base4')
# Map1

In [78]:
start_date = '2020-10-01'
end_date = '2021-03-31'
imgbase5 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filter(ee.Filter.eq('WRS_PATH', 232)) \
    .filter(ee.Filter.eq('WRS_ROW', 91)) \
    .filterDate(start_date, end_date) 

In [79]:
imgbase5 = find_closest(imgbase, imgbase5,28).sort('CLOUD_COVER').first()
imgbase5 = ee.Image(imgbase5)

In [80]:
# reproj5= imgbase5 \
#     .reproject(crs='EPSG:4326',crsTransform= [0.00025, 0, -180, 0, -0.00025, 80])

In [81]:
# Map1.addLayer(imgbase5, vizParams, 'img base5')
# Map1

In [82]:
start_date = '2020-10-01'
end_date = '2021-03-31'
imgbase6 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filter(ee.Filter.eq('WRS_PATH', 233)) \
    .filter(ee.Filter.eq('WRS_ROW', 91)) \
    .filterDate(start_date, end_date) 

In [83]:
imgbase6 = find_closest(imgbase5, imgbase6,360).sort('CLOUD_COVER').first()
imgbase6 = ee.Image(imgbase6)

In [84]:
# reproj6= imgbase6 \
#     .reproject(crs='EPSG:4326',crsTransform= [0.00025, 0, -180, 0, -0.00025, 80])

In [85]:
# Map1.addLayer(imgbase6, vizParams, 'img base6')
# Map1

In [86]:
# imgbase2.get('UTM_ZONE').getInfo()

In [87]:
# imgbase3.get('UTM_ZONE').getInfo()

In [88]:
# imgbase4.get('UTM_ZONE').getInfo()

In [89]:
# imgbase5.get('UTM_ZONE').getInfo()

In [90]:
# imgbase6.get('UTM_ZONE').getInfo()

In [91]:
imgbase = addNDVI(imgbase)
imgbase2 = addNDVI(imgbase2)
imgbase3 = addNDVI(imgbase3)
imgbase4 = addNDVI(imgbase4)
imgbase5 = addNDVI(imgbase5)
imgbase6 = addNDVI(imgbase6)

In [92]:
imgbase = addNDWI(imgbase)
imgbase2 = addNDWI(imgbase2)
imgbase3 = addNDWI(imgbase3)
imgbase4 = addNDWI(imgbase4)
imgbase5 = addNDWI(imgbase5)
imgbase6 = addNDWI(imgbase6)

In [93]:
imgbase = addNDGI(imgbase)
imgbase2 = addNDGI(imgbase2)
imgbase3 = addNDGI(imgbase3)
imgbase4 = addNDGI(imgbase4)
imgbase5 = addNDGI(imgbase5)
imgbase6 = addNDGI(imgbase6)

In [94]:
imgbase = addSAVI(imgbase)
imgbase2 = addSAVI(imgbase2)
imgbase3 = addSAVI(imgbase3)
imgbase4 = addSAVI(imgbase4)
imgbase5 = addSAVI(imgbase5)
imgbase6 = addSAVI(imgbase6)

In [95]:
imgbase = addNDSI(imgbase)
imgbase2 = addNDSI(imgbase2)
imgbase3 = addNDSI(imgbase3)
imgbase4 = addNDSI(imgbase4)
imgbase5 = addNDSI(imgbase5)
imgbase6 = addNDSI(imgbase6)

In [96]:
imgbase = addDate(imgbase)
imgbase2 = addDate(imgbase2)
imgbase3 = addDate(imgbase3)
imgbase4 = addDate(imgbase4)
imgbase5 = addDate(imgbase5)
imgbase6 = addDate(imgbase6)

In [97]:
imgbase = addCloud(imgbase)
imgbase2 = addCloud(imgbase2)
imgbase3 = addCloud(imgbase3)
imgbase4 = addCloud(imgbase4)
imgbase5 = addCloud(imgbase5)
imgbase6 = addCloud(imgbase6)

In [98]:
# imgbase.get('DATE_ACQUIRED').getInfo()

In [99]:
# imgbase2.get('DATE_ACQUIRED').getInfo()

In [100]:
# imgbase3.get('DATE_ACQUIRED').getInfo()

In [101]:
# imgbase4.get('DATE_ACQUIRED').getInfo()

In [102]:
# imgbase5.get('DATE_ACQUIRED').getInfo()

In [103]:
# imgbase6.get('DATE_ACQUIRED').getInfo()

In [104]:
imgbase_bandas = imgbase.select(['B1','B2','B3','B4','B5','B6','B7'])
imgbase_bandas2 = imgbase2.select(['B1','B2','B3','B4','B5','B6','B7'])
imgbase_bandas3 = imgbase3.select(['B1','B2','B3','B4','B5','B6','B7'])
imgbase_bandas4 = imgbase4.select(['B1','B2','B3','B4','B5','B6','B7'])
imgbase_bandas5 = imgbase5.select(['B1','B2','B3','B4','B5','B6','B7'])
imgbase_bandas6 = imgbase6.select(['B1','B2','B3','B4','B5','B6','B7'])

In [105]:
colec_base = ee.ImageCollection([imgbase,imgbase2,imgbase3,imgbase4,imgbase5,imgbase6]).mosaic()

In [106]:
# colec_base_r = ee.ImageCollection([reproj,reproj2,reproj3,reproj4,reproj5,reproj6]).mosaic()

In [107]:
Map1.addLayer(resampled, vizParams, 'img base r')
Map1.addLayer(imgbase, vizParams, 'img base')
# Map1.addLayer(imgbase_bandas2, vizParams, 'img base2')
# Map1.addLayer(imgbase_bandas3, vizParams, 'img base3')
# Map1.addLayer(imgbase_bandas4, vizParams, 'img base4')
# Map1.addLayer(imgbase_bandas5, vizParams, 'img base5')
# Map1.addLayer(imgbase_bandas6, vizParams, 'img base6')
Map1.addLayer(colec_base, vizParams, 'coleccion')
# Map1.addLayer(colec_base_r, vizParams, 'coleccion r')
Map1

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Indices por colección

### Índices 2021

In [142]:
# NDVI_2021 = NDVI_2021_sn.select('NDVI')
# NDWI_2021 = NDWI_2021_sn.select('NDWI')
# SAVI_2021 = SAVI_2021_sn.select('SAVI')
# NDGI_2021 = NDGI_2021_sn.select('NDGI')
# NDSI_2021 = NDSI_2021_sn.select('NDSI')

In [143]:
# Map9.addLayer(NDVI_2021, {'palette': color_terrain}, 'QMsn de NDVI 2021')
# Map9.addLayer(NDWI_2021, {'palette': color_ndwi}, 'QMsn de NDWI 2021')
# Map9.addLayer(SAVI_2021, {'palette': color_ndvi}, 'QMsn de SAVI 2021')
# Map9.addLayer(NDGI_2021, {'palette': color_glaciar}, 'QMsn de NDGI 2021')
# Map9.addLayer(NDSI_2021, {'palette': color_ndwi}, 'QMsn de NDSI 2021')
# Map9

### Índices 2020

In [144]:
# NDVI_2020 = NDVI_2020_sn.select('NDVI')
# NDWI_2020 = NDWI_2020_sn.select('NDWI')
# SAVI_2020 = SAVI_2020_sn.select('SAVI')
# NDGI_2020 = NDGI_2020_sn.select('NDGI')
# NDSI_2020 = NDSI_2020_sn.select('NDSI')

In [145]:
# Map5.addLayer(NDVI_2020, {'palette': color_terrain}, 'QMsn de NDVI 2020')
# Map5.addLayer(NDWI_2020, {'palette': color_ndwi}, 'QMsn de NDWI 2020')
# Map5.addLayer(SAVI_2020, {'palette': color_ndvi}, 'QMsn de SAVI 2020')
# Map5.addLayer(NDGI_2020, {'palette': color_glaciar}, 'QMsn de NDGI 2020')
# Map5.addLayer(NDSI_2020, {'palette': color_ndwi}, 'QMsn de NDSI 2020')
# Map5

### Índices 2019

In [146]:
# NDVI_2019 = NDVI_2019_sn.select('NDVI')
# NDWI_2019 = NDWI_2019_sn.select('NDWI')
# SAVI_2019 = SAVI_2019_sn.select('SAVI')
# NDGI_2019 = NDGI_2019_sn.select('NDGI')
# NDSI_2019 = NDSI_2019_sn.select('NDSI')

In [147]:
# Map6.addLayer(NDVI_2019, {'palette': color_terrain}, 'QMsn de NDVI 2019')
# Map6.addLayer(NDWI_2019, {'palette': color_ndwi}, 'QMsn de NDWI 2019')
# Map6.addLayer(SAVI_2019, {'palette': color_ndvi}, 'QMsn de SAVI 2019')
# Map6.addLayer(NDGI_2019, {'palette': color_glaciar}, 'QMsn de NDGI 2019')
# Map6.addLayer(NDSI_2019, {'palette': color_ndwi}, 'QMsn de NDSI 2019')
# Map6

### Índices 2018

In [148]:
# NDVI_2018 = NDVI_2018_sn.select('NDVI')
# NDWI_2018 = NDWI_2018_sn.select('NDWI')
# SAVI_2018 = SAVI_2018_sn.select('SAVI')
# NDGI_2018 = NDGI_2018_sn.select('NDGI')
# NDSI_2018 = NDSI_2018_sn.select('NDSI')

In [149]:
# Map6.addLayer(NDVI_2018, {'palette': color_terrain}, 'QMsn de NDVI 2018')
# Map6.addLayer(NDWI_2018, {'palette': color_ndwi}, 'QMsn de NDWI 2018')
# Map6.addLayer(SAVI_2018, {'palette': color_ndvi}, 'QMsn de SAVI 2018')
# Map6.addLayer(NDGI_2018, {'palette': color_glaciar}, 'QMsn de NDGI 2018')
# Map6.addLayer(NDSI_2018, {'palette': color_ndwi}, 'QMsn de NDSI 2018')
# Map6

## Matching de Histogramas

In [150]:
def lookup(source_hist, target_hist):
    """Creates a lookup table to make a source histogram match a target histogram.

    Args:
        source_hist: The histogram to modify. 
        Expects the Nx2 array format produced by ee.Reducer.autoHistogram.
        target_hist: The histogram to match to. 
        Expects the Nx2 array format produced by ee.Reducer.autoHistogram.

    Returns:
        A dictionary with 'x' and 'y' properties that respectively represent the x and y
        array inputs to the ee.Image.interpolate function.
    """

    # Split the histograms by column and normalize the counts.
    source_values = source_hist.slice(1, 0, 1).project([0])
    source_counts = source_hist.slice(1, 1, 2).project([0])
    source_counts = source_counts.divide(source_counts.get([-1]))

    target_values = target_hist.slice(1, 0, 1).project([0])
    target_counts = target_hist.slice(1, 1, 2).project([0])
    target_counts = target_counts.divide(target_counts.get([-1]))

    # Find first position in target where targetCount >= srcCount[i], for each i.
    def make_lookup(n):
        return target_values.get(target_counts.gte(n).argmax())

    lookup = source_counts.toList().map(make_lookup)

    return {'x': source_values.toList(), 'y': lookup}

In [151]:
def histogram_match(source_img, target_img, geometry):
    """Performs histogram matching for 3-band RGB images by forcing the histogram CDF of source_img to match target_img.

    Args:
        source_img: A 3-band ee.Image to be color matched. Must have bands named 'R', 'G', and 'B'.
        target_img: A 3-band ee.Image for color reference. Must have bands named 'R', 'G', and 'B'.
        geometry: An ee.Geometry that defines the region to generate RGB histograms for.
          It should intersect both source_img and target_img inputs.

    Returns:
        A copy of src_img color-matched to target_img.
    """

    args = {
        'reducer': ee.Reducer.autoHistogram(**{'maxBuckets': 1024, 'cumulative': True}),
        'geometry': geometry,
        'scale': 1, # Need to specify a scale, but it doesn't matter what it is because bestEffort is true.
        'maxPixels': 1048576 * 4 - 1,
        'bestEffort': True
    }

    # Only use pixels in target that have a value in source (inside the footprint and unmasked).
    source = source_img.reduceRegion(**args)
    target = target_img.updateMask(source_img.mask()).reduceRegion(**args)

    return ee.Image.cat(
        source_img.select(['B1']).interpolate(**lookup(source.getArray('B1'), target.getArray('B1'))),
        source_img.select(['B2']).interpolate(**lookup(source.getArray('B2'), target.getArray('B2'))),
        source_img.select(['B3']).interpolate(**lookup(source.getArray('B3'), target.getArray('B3'))),
        source_img.select(['B4']).interpolate(**lookup(source.getArray('B4'), target.getArray('B4'))),
        source_img.select(['B5']).interpolate(**lookup(source.getArray('B5'), target.getArray('B5'))),
        source_img.select(['B6']).interpolate(**lookup(source.getArray('B6'), target.getArray('B6'))),
        source_img.select(['B7']).interpolate(**lookup(source.getArray('B7'), target.getArray('B7'))),
        source_img.select(['B8']).interpolate(**lookup(source.getArray('B8'), target.getArray('B8'))),
        source_img.select(['B9']).interpolate(**lookup(source.getArray('B9'), target.getArray('B9'))),
        source_img.select(['B10']).interpolate(**lookup(source.getArray('B10'), target.getArray('B10'))),
        source_img.select(['B11']).interpolate(**lookup(source.getArray('B11'), target.getArray('B11'))),
    ).copyProperties(source_img, ['system:time_start'])

In [152]:
geometry = geometria_base2.geometry()

In [153]:
# geometry = ee.Geometry({
#   'type': 'Polygon',
#   'coordinates':
#     [[[-72.674, -45.045],
#       [-71.603, -45.086],
#       [-71.554, -46.027],
#       [-72.674, -45.045]]]
# });

In [154]:
# geometryCoordinates = geometry.coordinates()

In [155]:
# print('geometry.coordinates(...) =', geometryCoordinates)

In [156]:
QM_NDVI_2021_m = histogram_match(NDVI_2021_sn,colec_base, geometry)

In [157]:
NDVI_tier1 = histogram_match(QM_NDVI_T1, colec_base, geometry)

In [158]:
NDVI_tier2 = histogram_match(QM_NDVI_T2, colec_base, geometry)

In [159]:
NDVI_tier3 = histogram_match(QM_NDVI_T3, colec_base, geometry)

In [160]:
NDVI_tier4 = histogram_match(QM_NDVI_T4, colec_base, geometry)

In [163]:
vis_params_refl = {'min': 0, 'max': 0.25} 

In [164]:
# import folium

# def add_ee_layer(self, ee_image_object, vis_params, name):
#   map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
#   folium.raster_layers.TileLayer(
#       tiles=map_id_dict['tile_fetcher'].url_format,
#       attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
#       name=name,
#       overlay=True,
#       control=True
#   ).add_to(self)

# folium.Map.add_ee_layer = add_ee_layer

In [165]:
# lon, lat, zoom = -45, -70 , 4
# vis_params_refl = {'min': 0, 'max': 0.25}
# map_matched = folium.Map(location=[lat, lon], zoom_start=zoom)
# map_matched.add_ee_layer(QM_NDVI_2021_m, vizParams, 'matched')
# display(map_matched.add_child(folium.LayerControl()))

## Generación de Mosaico

In [59]:
NDVI_2021_landsat = NDVI_2021.set('SENSOR_ID', 'OLI_TIRS')

In [60]:
cloud_2021 = ee.Algorithms.Landsat.simpleCloudScore(NDVI_2021_landsat)

In [61]:
cloud_2021 = cloud_2021.select('cloud')

In [62]:
palette_nubes = ['#1a9850', '#66bd63', '#a6d96a', '#d9ef8b', '#fee08b', '#fdae61', '#f46d43', '#d73027']

In [63]:
resultados= [cloud_2021, NDVI_2021]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes 2021',
    'QM NDVI 2021']

In [64]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [65]:
def llenado(image):
    return image.where(cloud_2021.gt(10),QM_NDVI_T1)

In [66]:
img1 = llenado(NDVI_2021)

In [67]:
img1_landsat = img1.set('SENSOR_ID', 'OLI_TIRS')
cloud_img1 = ee.Algorithms.Landsat.simpleCloudScore(img1_landsat)
cloud_img1 = cloud_img1.select('cloud')

In [68]:
resultados= [cloud_img1, img1]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 1',
    'Mosaico 1']

In [69]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [70]:
def llenado2(image):
    return image.where(cloud_img1.gt(10), QM_NDVI_T2)

In [71]:
img2 = llenado2(img1)

In [72]:
img2_landsat = img2.set('SENSOR_ID', 'OLI_TIRS')
cloud_img2 = ee.Algorithms.Landsat.simpleCloudScore(img2_landsat)
cloud_img2 = cloud_img2.select('cloud')

In [73]:
resultados= [cloud_img2, img2]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 2',
    'Mosaico 2']

In [74]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [79]:
def llenado3(image):
    return image.where(cloud_img2.gt(10), QM_NDVI_T3)

In [80]:
img3 = llenado3(img2)

In [81]:
img3_landsat = img3.set('SENSOR_ID', 'OLI_TIRS')
cloud_img3 = ee.Algorithms.Landsat.simpleCloudScore(img3_landsat)
cloud_img3 = cloud_img3.select('cloud')

In [82]:
resultados= [cloud_img3, img3]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 3',
    'Mosaico 3']

In [83]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [84]:
def llenado4(image):
    return image.where(cloud_img3.gt(10), QM_NDVI_T4)

In [85]:
img4 = llenado4(img3)

In [86]:
img4_landsat = img4.set('SENSOR_ID', 'OLI_TIRS')
cloud_img4 = ee.Algorithms.Landsat.simpleCloudScore(img4_landsat)
cloud_img4 = cloud_img4.select('cloud')

In [87]:
resultados= [cloud_img4, img4]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 4',
    'Mosaico 4']

In [88]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [89]:
hansenImage = ee.Image('UMD/hansen/global_forest_change_2015')
datamask = hansenImage.select('datamask')

In [93]:
def llenado5(image):
    return image.where(datamask.eq(2), QM_NDWI_T1)

In [94]:
img5 = llenado5(img4)

In [95]:
img5_landsat = img5.set('SENSOR_ID', 'OLI_TIRS')

In [96]:
resultados= [img4, img5]
labels = [
    'Mosaico 4',
    'Mosaico 5']
vis_params3=  [
     vizParams, 
    vizParams]

In [97]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params3, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [98]:
Map10.addLayer(img4, vizParams, 'Mosaico 5')
Map10.addLayer(img5, vizParams, 'Mosaico 6')
Map10

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Resultado Final

In [204]:
# img7 = img6.select(['B1','B2','B3','B4','B5','B6','B7'])
# Map10.addLayer(img7, vizParams, 'Mosaico 7')
# Map10

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [205]:
# type(img6)

In [206]:
# img6.bandNames().getInfo()

In [207]:
colec7 = ee.ImageCollection(img7)

In [208]:
coleccion7 = colec7.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [209]:
property = 'date'

In [210]:
options = {"title": 'Fechas Greenest 2021',
          "xlabel": 'Date',
          "ylabel": 'Pixel count',
          "colors": ['#1d6b99']
          }

In [211]:
# muestra = img6.sample(patagonia_grande, 10000)

In [212]:
# chart.feature_histogram(muestra, property, **options)

## Mosaicos Matched

In [213]:
NDVI_2021_landsat_m = QM_NDVI_2021_m.set('SENSOR_ID', 'OLI_TIRS')

In [214]:
cloud_2021_m = ee.Algorithms.Landsat.simpleCloudScore(NDVI_2021_landsat_m)

In [215]:
cloud_2021_m = cloud_2021_m.select('cloud')

In [216]:
palette_nubes = ['#1a9850', '#66bd63', '#a6d96a', '#d9ef8b', '#fee08b', '#fdae61', '#f46d43', '#d73027']

In [217]:
QM_NDVI_2021_m = ee.Image(QM_NDVI_2021_m)

In [218]:
resultados= [cloud_2021_m, QM_NDVI_2021_m]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes 2021',
    'QM NDVI 2021']

In [219]:
# geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
#                    ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [220]:
def llenado(image):
    return image.where(cloud_2021_m.gt(20),NDVI_2021_sn_m)

In [221]:
img1_m = llenado(QM_NDVI_2021_m)

In [222]:
img1_landsat_m = img1_m.set('SENSOR_ID', 'OLI_TIRS')
cloud_img1_m = ee.Algorithms.Landsat.simpleCloudScore(img1_landsat_m)
cloud_img1_m = cloud_img1_m.select('cloud')

In [223]:
img1_m = ee.Image(img1_m)

In [224]:
resultados= [cloud_img1_m, img1_m]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 1',
    'Mosaico 1']

In [225]:
# geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
#                    ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [226]:
def llenado2(image):
    return image.where(cloud_img1_m.gt(20), NDVI_2020_sn_m)

In [227]:
img2_m = llenado2(img1_m)

In [228]:
img2_landsat_m = img2_m.set('SENSOR_ID', 'OLI_TIRS')
cloud_img2_m = ee.Algorithms.Landsat.simpleCloudScore(img2_landsat_m)
cloud_img2_m = cloud_img2_m.select('cloud')

In [229]:
img2_m = ee.Image(img2_m)

In [230]:
resultados= [cloud_img2_m, img2_m]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 2',
    'Mosaico 2']

In [231]:
# geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
#                    ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [232]:
def llenado3(image):
    return image.where(cloud_img2_m.gt(20), NDVI_2019_sn_m)

In [233]:
img3_m = llenado3(img2_m)

In [234]:
img3_landsat_m = img3_m.set('SENSOR_ID', 'OLI_TIRS')
cloud_img3_m = ee.Algorithms.Landsat.simpleCloudScore(img3_landsat_m)
cloud_img3_m = cloud_img3_m.select('cloud')

In [235]:
img3_m = ee.Image(img3_m)

In [236]:
resultados= [cloud_img3_m, img3_m]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 3',
    'Mosaico 3']

In [237]:
# geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
#                    ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [238]:
def llenado4(image):
    return image.where(cloud_img3_m.gt(20), NDVI_2018_sn_m)

In [239]:
img4_m = llenado4(img3_m)

In [240]:
img4_landsat_m = img4_m.set('SENSOR_ID', 'OLI_TIRS')
cloud_img4_m = ee.Algorithms.Landsat.simpleCloudScore(img4_landsat_m)
cloud_img4_m = cloud_img4_m.select('cloud')

In [241]:
img4_m = ee.Image(img4_m)

In [242]:
resultados= [cloud_img4_m, img4_m]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 4',
    'Mosaico 4']

In [243]:
# geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
#                    ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [244]:
def llenado5(image):
    return image.where(cloud_img4_m.gt(20), NDVI_2017_sn_m)

In [245]:
img5_m = llenado5(img4_m)

In [246]:
img5_landsat_m = img5_m.set('SENSOR_ID', 'OLI_TIRS')
cloud_img5_m = ee.Algorithms.Landsat.simpleCloudScore(img5_landsat_m)
cloud_img5_m = cloud_img5_m.select('cloud')

In [247]:
img5_m = ee.Image(img5_m)

In [248]:
resultados= [cloud_img5_m, img5_m]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 5',
    'Mosaico 5']

In [249]:
# geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
#                    ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [250]:
hansenImage = ee.Image('UMD/hansen/global_forest_change_2015')
datamask = hansenImage.select('datamask')

In [251]:
def llenado6(image):
    return image.where(datamask.eq(2), NDWI_2021_sur_sn_m)

In [252]:
img6_m = llenado6(img5_m)

In [253]:
# img6_m.bandNames().getInfo()

In [254]:
img6_m = ee.Image(img6_m)

In [255]:
resultados= [img5_m, img6_m]
labels = [
    'Mosaico 5',
    'Mosaico 6']
vis_params3=  [
     vizParams, 
    vizParams]

In [256]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params3, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [257]:
img6_m_bandas = img6_m.select(['B1','B2','B3','B4','B5','B6','B7'])

In [258]:
vizParams2 = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.2,
  'gamma': [1, 1.1, 1]
}

### Resultado Final

In [259]:
Map10.addLayer(img6_m_bandas, vizParams, 'Mosaico 6m')
Map10

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [260]:
# Map12.addLayer(img6_m_bandas, vizParams2, 'Mosaico 6m')
# Map12

In [261]:
# img6_m.bandNames().getInfo()

## Clasificación

In [262]:
val_shp = os.path.join(work_dir, 'validacion.shp')
merged_original = os.path.join(work_dir,'datos_1.shp')

In [263]:
val = geemap.shp_to_ee(val_shp)
# Map13.addLayer(val, {}, 'Áreas de Validación')

In [264]:
mergedoriginal = geemap.shp_to_ee(merged_original)
# Map13.addLayer( mergedoriginal, {}, 'Áreas de entrenamiento Original')

In [265]:
# img6_m = addNDVI(img6_m)
# img6_m = addNDWI(img6_m)
# img6_m = addNDGI(img6_m)
# img6_m = addSAVI(img6_m)
# img6_m = addNDSI(img6_m)
# img6_m = addDate(img6_m)
# img6_m = addCloud(img6_m)

In [266]:
dataset = img6.sampleRegions(**{
    'collection': mergedoriginal,
    'properties': ['ID'],
    'scale': 30,
})

In [267]:
datasetval = img6.sampleRegions(**{
    'collection': val,
    'properties': ['ID'],
    'scale': 10,
})

In [268]:
datasetMatch = img6_m.sampleRegions(**{
    'collection': mergedoriginal,
    'properties': ['ID'],
    'scale': 30,
})

In [269]:
datasetvalMatch= img6_m.sampleRegions(**{
    'collection': val,
    'properties': ['ID'],
    'scale': 10,
})

In [270]:
# print(dataset.size().getInfo())

In [271]:
# print(dataset.first().getInfo())

In [272]:
# print(datasetval.first().getInfo())

### Muestras y datos de validación

In [273]:
# Guardamos las bandas

# Esta propiedad guarda las clases de landcover
# label = 'b1'
label = 'IDH'
label1 = 'ID'
# Overlay the points on the imagery to get training.
# sample = landsat.select(bands).sampleRegions(**{
#   'collection': points,
#   'properties': [label],
#   'scale': 30
# })

# # Adds a column of deterministic pseudorandom numbers. 
# sample = sample.randomColumn()

# split = 0.7 

# training = sample.filter(ee.Filter.lt('random', split))
# validation = sample.filter(ee.Filter.gte('random', split))
bands = ['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'NDVI',
 'NDWI',
 'NDGI',
 'SAVI',
 'NDSI',
]
bands1 = ['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11']

### Entrenando el modelo

In [274]:
classifier = ee.Classifier.smileRandomForest(100).train(dataset, label1, bands)
classifierMatch = ee.Classifier.smileRandomForest(100).train(datasetMatch, label1, bands1)

In [275]:
result1 = img6.select(bands).classify(classifier)
result2_M = img6_m.select(bands1).classify(classifierMatch)

### Categorizando

In [276]:
class_values1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

class_palette1 = ['6bee8c', # 1
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 'dc07e7',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '023208',
 '03c2ff',
 '97ad0b',
 'a4f8ff',
 'c0c233',               
]

In [277]:
landcover1 = result1.set('classification_class_values', class_values1)
landcover1 = landcover1.set('classification_class_palette', class_palette1)

In [278]:
landcoverMatch = result2_M.set('classification_class_values', class_values1)
landcoverMatch = landcoverMatch.set('classification_class_palette', class_palette1)

In [279]:
Map14.addLayer(landcover1, {}, 'Land cover clasificado Random Forest' )
Map14.addLayer(landcoverMatch, {}, 'Land cover clasificado Random Forest Match' )

In [280]:
type(landcover1)

ee.image.Image

In [281]:

legend_dict = {
    '01 Achaparrado': '6bee8c',
    '02 Agrícola': 'f0ff00',
    '03 Agua': '0c40f1',
    '04 Arenal o Roca': '685205',
    '05 Bosque Degradado': '0d4404',
    '06 Cuncunas': 'e82d13',
    '07 Estepa': 'f99504',
    '08 Humedal': '17d4c6',
    '09 Mallin': 'dc07e7',
    '10 Matorral abierto': '24ff00',
    '11 Matorral denso': '2de50f',
    '12 Plantaciones': 'ccba7d',
    '13 Praderas': 'dec61a',
    '14 Primario': '023208',
    '15 Ríos': '03c2ff',
    '16 Secundario': '97ad0b',
    '17 Glaciar': 'a4f8ff',
    '18 Suelo desnudo': 'c0c233',
}
Map14.add_legend(legend_title="Random Forest", legend_dict=legend_dict)
Map14

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Exportando

In [282]:
# landcover_resampleado = landcover1.resample('bicubic')

In [283]:
# print('es(...)', ee.Geometry(landcover1).isUnbounded())

In [284]:
# landcover1 = landcover1.set('Geometry',patagonia.geometries())
# landcover1 = ee.Image(landcover1)

In [285]:
work_dir = os.path.expanduser('../../../Data/shapes/')
corte = os.path.join(work_dir, 'img_base.shp')
corte = geemap.shp_to_ee(corte)

In [286]:
clipped = landcover1.clip(corte)

In [287]:
# Map15.addLayer(clipped,{},'corte')
# Map15

In [288]:
# type(clipped)

In [289]:
# geemap.ee_export_image_to_drive(landcover1, description='pixel_fix', folder='export', region=patagonia_grande.geometry(), scale=40)

In [290]:
# geemap.ee_export_image_to_drive(img7, description='mosaico_final', folder='export', region=patagonia_grande.geometry(), scale=40)

In [291]:
# geemap.ee_export_image_to_drive(img6_m_bandas, description='mosaico_final_match', folder='export', region=patagonia_grande.geometry(), scale=40)